In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [2]:
me = pd.read_excel("../../../Features/MacroEcon/US_FEATURES_ME.xlsx",sheet_name="in",index_col=0)

FileNotFoundError: [Errno 2] No such file or directory: '../../../Features/MacroEcon/US_FEATURES_ME.xlsx'

In [ ]:
me.drop(columns=["GDP","GDP_P","GDP_PC"],inplace=True)

In [ ]:
me.index

In [ ]:
fd = pd.read_csv("../../../Features/Fundamental/daily_financial_ratios_new.csv",index_col=0,parse_dates=[0])

In [ ]:
fd.index

In [101]:
mkt = pd.read_csv("../../../Features/Market/market_data_close_plus_VIX_INDEX.csv",index_col=0,parse_dates=[0])

In [103]:
mkt.drop(columns=["^FTSE_Close","^FTSE_Volume","^VIX_Volume"],inplace=True)

In [104]:
mkt = mkt.pct_change()

In [105]:
mkt

,TMUS_Close,TMUS_Volume,TSLA_Close,TSLA_Volume,MU_Close,MU_Volume,FITB_Close,FITB_Volume,MRNA_Close,MRNA_Volume,...,AAPL_Volume,PEP_Close,PEP_Volume,BABA_Close,BABA_Volume,^VIX_Close,^IXIC_Close,^IXIC_Volume,AAL_Close,AAL_Volume
Date,,,,,,,,,,,,,,,,,,,,,
2016-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-10-03,0.003425,-0.447305,0.047395,1.319878,-0.002812,0.257126,-0.005376,-0.004888,NaN,NaN,...,-0.403454,-0.004781,-0.363473,-0.003876,-0.303515,0.021068,-0.002095,-0.221254,0.029773,0.195119
2016-10-04,-0.002773,0.659567,-0.010716,-0.409740,0.003948,1.029438,0.005897,0.534632,NaN,NaN,...,0.370246,-0.010624,0.276851,0.000664,0.020213,0.004422,-0.002115,0.065415,-0.001857,0.021027
2016-10-05,-0.014545,-0.320605,-0.013954,-0.469857,-0.005618,0.242863,0.013190,-0.225130,NaN,NaN,...,-0.278567,-0.010177,0.080084,0.014320,0.190418,-0.046955,0.004983,0.025951,0.006378,-0.480112
2016-10-06,-0.012372,0.295526,-0.035786,1.505140,0.001695,-0.681031,0.000000,-0.218909,NaN,NaN,...,0.341498,0.003679,-0.224075,0.001122,-0.249755,-0.011547,-0.001725,-0.056646,0.007922,0.094863
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-10-08,-0.015912,0.100171,-0.010232,-0.129440,-0.005954,-0.103642,0.002281,-0.288267,-0.014384,-0.327433,...,-0.048823,-0.002302,-0.091008,0.035385,-0.273884,-0.039406,-0.005083,-0.096179,-0.004958,-0.258246
2021-10-11,-0.033245,0.654038,0.008211,-0.150247,-0.012835,-0.070059,-0.010466,0.208986,0.000623,-0.281974,...,0.097644,0.001346,-0.345891,0.015045,0.262246,0.065530,-0.006402,-0.009950,0.002990,0.092993
2021-10-12,-0.007765,-0.282147,0.017400,0.550671,-0.036117,1.906387,-0.004599,0.233543,0.019764,0.336673,...,0.133179,0.004416,0.167014,-0.005794,-0.419042,-0.007500,-0.001400,0.146157,0.007948,-0.219835


In [106]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    display(mkt.iloc[0:100])

,TMUS_Close,TMUS_Volume,TSLA_Close,TSLA_Volume,MU_Close,MU_Volume,FITB_Close,FITB_Volume,MRNA_Close,MRNA_Volume,^GSPC_Close,^GSPC_Volume,SBUX_Close,SBUX_Volume,AAPL_Close,AAPL_Volume,PEP_Close,PEP_Volume,BABA_Close,BABA_Volume,^VIX_Close,^IXIC_Close,^IXIC_Volume,AAL_Close,AAL_Volume
Date,,,,,,,,,,,,,,,,,,,,,,,,,
2016-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-10-03,0.003425,-0.447305,0.047395,1.319878,-0.002812,0.257126,-0.005376,-0.004888,NaN,NaN,-0.003261,-0.248192,-0.005541,-0.602072,-0.004688,-0.403454,-0.004781,-0.363473,-0.003876,-0.303515,0.021068,-0.002095,-0.221254,0.029773,0.195119
2016-10-04,-0.002773,0.659567,-0.010716,-0.409740,0.003948,1.029438,0.005897,0.534632,NaN,NaN,-0.004956,0.195484,-0.005758,0.127368,0.004266,0.370246,-0.010624,0.276851,0.000664,0.020213,0.004422,-0.002115,0.065415,-0.001857,0.021027
2016-10-05,-0.014545,-0.320605,-0.013954,-0.469857,-0.005618,0.242863,0.013190,-0.225130,NaN,NaN,0.004297,0.041499,-0.003363,0.206431,0.000443,-0.278567,-0.010177,0.080084,0.014320,0.190418,-0.046955,0.004983,0.025951,0.006378,-0.480112
2016-10-06,-0.012372,0.295526,-0.035786,1.505140,0.001695,-0.681031,0.000000,-0.218909,NaN,NaN,0.000482,-0.113911,-0.003936,-0.177296,0.007430,0.341498,0.003679,-0.224075,0.001122,-0.249755,-0.011547,-0.001725,-0.056646,0.007922,0.094863
2016-10-07,-0.005055,-0.045564,-0.021841,-0.257346,-0.006768,0.103773,-0.002411,0.074625,NaN,NaN,-0.003253,0.045743,0.006022,0.187495,0.001493,-0.153614,-0.003853,-0.099795,-0.010086,-0.094198,0.049844,-0.002723,-0.004456,-0.011527,-0.005232
2016-10-10,0.019439,-0.240606,0.022074,-0.050587,-0.006246,-0.209292,-0.000483,0.081264,NaN,NaN,0.004606,-0.194299,-0.002993,-0.007610,0.017447,0.487618,0.002170,-0.290810,0.022736,0.286139,-0.007418,0.006853,-0.171560,0.030745,0.351964
2016-10-11,-0.011701,-0.108172,-0.004230,-0.297892,-0.036571,1.018825,-0.014023,0.138888,NaN,NaN,-0.012446,0.178883,-0.007129,0.345514,0.002154,0.767331,-0.006119,0.222838,-0.029333,0.295226,0.147982,-0.015366,0.335396,-0.017742,0.445943
2016-10-12,0.009208,-0.297669,0.007046,-0.153625,-0.004745,-0.529769,-0.000981,-0.138880,NaN,NaN,0.001147,-0.134129,0.004535,-0.349770,0.008942,-0.413082,0.004831,-0.103450,-0.015300,0.235871,0.035807,-0.001481,-0.140387,-0.021466,-0.261562


In [107]:
dataset = mkt.join(fd,how="outer").join(me,how="outer")

In [109]:
dataset = dataset.loc[dataset["^GSPC_Close"].notna()]

In [115]:
dataset = dataset.loc[dataset.index<="2021-09-30"]

In [116]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1284 entries, 2016-10-03 to 2021-09-30
Columns: 102 entries, TMUS_Close to GDP_PC_P
dtypes: float64(102)
memory usage: 1.0 MB


In [94]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    display(dataset.iloc[0:100])

,TMUS_Close,TMUS_Volume,TSLA_Close,TSLA_Volume,MU_Close,MU_Volume,FITB_Close,FITB_Volume,MRNA_Close,MRNA_Volume,^GSPC_Close,^GSPC_Volume,SBUX_Close,SBUX_Volume,AAPL_Close,AAPL_Volume,PEP_Close,PEP_Volume,BABA_Close,BABA_Volume,^VIX_Close,^VIX_Volume,^IXIC_Close,^IXIC_Volume,AAL_Close,AAL_Volume,AAPL_currentRatio,AAPL_operatingProfitMargin,AAPL_netProfitMargin,AAPL_debtEquityRatio,AAPL_priceToBookRatio,AAPL_enterpriseValueMultiple,AAPL_priceFairValue,MRNA_currentRatio,MRNA_operatingProfitMargin,MRNA_netProfitMargin,MRNA_debtEquityRatio,MRNA_priceToBookRatio,MRNA_enterpriseValueMultiple,MRNA_priceFairValue,AAL_currentRatio,AAL_operatingProfitMargin,AAL_netProfitMargin,AAL_debtEquityRatio,AAL_priceToBookRatio,AAL_enterpriseValueMultiple,AAL_priceFairValue,TSLA_currentRatio,TSLA_operatingProfitMargin,TSLA_netProfitMargin,TSLA_debtEquityRatio,TSLA_priceToBookRatio,TSLA_enterpriseValueMultiple,TSLA_priceFairValue,MU_currentRatio,MU_operatingProfitMargin,MU_netProfitMargin,MU_debtEquityRatio,MU_priceToBookRatio,MU_enterpriseValueMultiple,MU_priceFairValue,SBUX_currentRatio,SBUX_operatingProfitMargin,SBUX_netProfitMargin,SBUX_debtEquityRatio,SBUX_priceToBookRatio,SBUX_enterpriseValueMultiple,SBUX_priceFairValue,TMUS_currentRatio,TMUS_operatingProfitMargin,TMUS_netProfitMargin,TMUS_debtEquityRatio,TMUS_priceToBookRatio,TMUS_enterpriseValueMultiple,TMUS_priceFairValue,FITB_currentRatio,FITB_operatingProfitMargin,FITB_netProfitMargin,FITB_debtEquityRatio,FITB_priceToBookRatio,FITB_enterpriseValueMultiple,FITB_priceFairValue,PEP_currentRatio,PEP_operatingProfitMargin,PEP_netProfitMargin,PEP_debtEquityRatio,PEP_priceToBookRatio,PEP_enterpriseValueMultiple,PEP_priceFairValue,BABA_currentRatio,BABA_operatingProfitMargin,BABA_netProfitMargin,BABA_debtEquityRatio,BABA_priceToBookRatio,BABA_enterpriseValueMultiple,BABA_priceFairValue,YC_L,YC_S,YC_C,YC_3M,YC_10Y,CPI,GDP_PC_P
2016-10-03,0.003425,-0.447305,0.047395,1.319878,-0.002812,0.257126,-0.005376,-0.004888,NaN,NaN,-0.003261,-0.248192,-0.005541,-0.602072,-0.004688,-0.403454,-0.004781,-0.363473,-0.003876,-0.303515,0.021068,NaN,-0.002095,-0.221254,0.029773,0.195119,1.311599,0.238562,0.184050,1.415043,4.017525,42.903582,4.017525,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.749749,0.174949,0.091672,10.839286,3.696319,22.008341,3.696319,1.381611,-0.187431,-0.230854,3.709352,11.790526,-496.392415,11.790526,2.246667,-0.010352,-0.074189,1.215558,1.114463,26.918614,1.114463,1.143934,0.195170,0.143967,1.417511,13.880394,59.450152,13.880394,1.631627,0.089677,0.024398,2.677379,2.045803,21.545158,2.045803,1.152718,0.0,0.233357,7.572580,0.796983,42.820573,0.796983,1.316188,0.176015,0.124290,4.863910,12.428761,51.583454,12.428761,1.858838,0.274118,0.234807,0.584903,4.980202,18.859033,4.980202,-1.962532,-0.608689,0.179411,0.32,1.63,0.001873,0.008485
2016-10-04,-0.002773,0.659567,-0.010716,-0.409740,0.003948,1.029438,0.005897,0.534632,NaN,NaN,-0.004956,0.195484,-0.005758,0.127368,0.004266,0.370246,-0.010624,0.276851,0.000664,0.020213,0.004422,NaN,-0.002115,0.065415,-0.001857,0.021027,1.311599,0.238562,0.184050,1.415043,4.017525,42.903582,4.017525,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.749749,0.174949,0.091672,10.839286,3.696319,22.008341,3.696319,1.381611,-0.187431,-0.230854,3.709352,11.790526,-496.392415,11.790526,2.246667,-0.010352,-0.074189,1.215558,1.114463,26.918614,1.114463,1.143934,0.195170,0.143967,1.417511,13.880394,59.450152,13.880394,1.631627,0.089677,0.024398,2.677379,2.045803,21.545158,2.045803,1.152718,0.0,0.233357,7.572580,0.796983,42.820573,0.796983,1.316188,0.176015,0.124290,4.863910,12.428761,51.583454,12.428761,1.858838,0.274118,0.234807,0.584903,4.980202,18.859033,4.980202,-1.806870,-0.715504,0.137798,0.33,1.69,0.001873,0.008485
2016-10-05,-0.014545,-0.320605,-0.013954,-0.469857,-0.005618,0.242863,0.013190,-0.225130,NaN,NaN,0.004297,0.041499,-0.003363,0.206431,0.000443,-0.278567,-0.010177,0.080084,0.014320,0.190418,-0.046955,NaN,0.004983,0.025951,0.006378,-0.480112,1.311599,0.238562,0.184050,1.415043,4.017525,42.

In [117]:
mkt.describe()

,TMUS_Close,TMUS_Volume,TSLA_Close,TSLA_Volume,MU_Close,MU_Volume,FITB_Close,FITB_Volume,MRNA_Close,MRNA_Volume,...,AAPL_Volume,PEP_Close,PEP_Volume,BABA_Close,BABA_Volume,^VIX_Close,^IXIC_Close,^IXIC_Volume,AAL_Close,AAL_Volume
count,1294.000000,1294.000000,1294.000000,1294.000000,1294.000000,1294.000000,1294.000000,1294.000000,732.000000,732.000000,...,1294.000000,1294.000000,1294.000000,1294.000000,1294.000000,1294.000000,1294.000000,1294.000000,1294.000000,1294.000000
mean,0.000872,0.113294,0.002976,0.080085,0.001441,0.063830,0.000896,0.057962,0.005179,0.159859,...,0.052077,0.000380,0.053426,0.000579,0.083571,0.003758,0.000883,0.019372,0.000134,0.082698
std,0.017240,0.630447,0.036385,0.499827,0.028455,0.404847,0.025018,0.392828,0.050268,0.786743,...,0.367807,0.013015,0.369622,0.021336,0.596098,0.089911,0.013330,0.313929,0.035613,0.499844
min,-0.112088,-0.901856,-0.185778,-0.738683,-0.198186,-0.681031,-0.217811,-0.726872,-0.179669,-0.836279,...,-0.712266,-0.114283,-0.747127,-0.133422,-0.647776,-0.259057,-0.123213,-0.823975,-0.252246,-0.744530
25%,-0.006954,-0.239981,-0.014266,-0.222895,-0.012862,-0.199424,-0.009499,-0.184918,-0.023631,-0.276828,...,-0.181136,-0.004726,-0.178206,-0.011275,-0.223118,-0.043453,-0.003701,-0.066892,-0.014967,-0.219811
50%,0.000856,-0.008767,0.001452,-0.037540,0.001191,-0.013968,0.000660,-0.007072,0.001015,-0.033763,...,-0.019150,0.000564,-0.006777,0.000525,-0.019310,-0.006053,0.001156,-0.003331,-0.000820,-0.036166
75%,0.008340,0.263846,0.019111,0.234533,0.017118,0.205226,0.010703,0.210106,0.030472,0.339818,...,0.194686,0.005812,0.202905,0.012543,0.242814,0.034582,0.007258,0.073968,0.013783,0.249913
max,0.117828,6.558763,0.198949,5.568608,0.133415,2.595862,0.185033,3.662435,0.278107,7.546809,...,2.628303,0.129366,2.682989,0.132919,12.219066,1.155979,0.093460,9.527690,0.410970,5.302531
